In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import matplotlib.pyplot as plt
plt.rc('figure', figsize = (10,6))
np.set_printoptions(precision = 4, suppress = True)

In [3]:
# Import two datasets
colnames = ["LOAN_ID", "ORIG_CHN", "Seller.Name", "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "ORIG_DTE","FRST_DTE", "OLTV", 
            "OCLTV", "NUM_BO", "DTI", "CSCORE_B", "FTHB_FLG", "PURPOSE", "PROP_TYP","NUM_UNIT", "OCC_STAT", 
            "STATE", "ZIP_3", "MI_PCT", "Product.Type", "CSCORE_C", "MI_TYPE", "RELOCATION_FLG"]
Data_A = pd.read_table('D:/1-Intern/Project_1/2018Q1/Acquisition_2018Q1.txt', 
                       sep = '|', header = None, names = colnames)
colnames_P = ["LOAN_ID", "Monthly.Rpt.Prd", "Servicer.Name", "LAST_RT", "LAST_UPB", "Loan.Age", "Months.To.Legal.Mat", 
              "Adj.Month.To.Mat", "Maturity.Date", "MSA", "Delq.Status", "MOD_FLAG", "Zero.Bal.Code", 
              "ZB_DTE", "LPI_DTE", "FCC_DTE","DISP_DT", "FCC_COST", "PP_COST", "AR_COST", "IE_COST", "TAX_COST", "NS_PROCS",
              "CE_PROCS", "RMW_PROCS", "O_PROCS", "NON_INT_UPB", "PRIN_FORG_UPB_FHFA", "REPCH_FLAG", "PRIN_FORG_UPB_OTH", "TRANSFER_FLG"]
Data_P = pd.read_table('D:/1-Intern/Project_1/2018Q1/Performance_2018Q1.txt', 
                       sep = '|', header = None, names = colnames_P,low_memory=False)

In [4]:
df = pd.merge(Data_A.loc[:,["LOAN_ID", "ORIG_RT", "ORIG_AMT", "ORIG_TRM", "OLTV", "DTI", "CSCORE_B", "FTHB_FLG","STATE" ]],
              Data_P.loc[:,["LOAN_ID", "Monthly.Rpt.Prd", "LAST_RT", "LAST_UPB","Delq.Status", "MOD_FLAG", "Zero.Bal.Code", "ZB_DTE", "FCC_DTE"]] , on = 'LOAN_ID')

In [11]:
# Convert string into datetime
from datetime import datetime
df['Monthly.Rpt.Prd'] = [datetime.strptime(x, '%m/%d/%Y') for x in df.loc[:,'Monthly.Rpt.Prd']]

In [12]:
df.dtypes

LOAN_ID                     int64
ORIG_RT                   float64
ORIG_AMT                    int64
ORIG_TRM                    int64
OLTV                        int64
DTI                       float64
CSCORE_B                  float64
FTHB_FLG                   object
STATE                      object
Monthly.Rpt.Prd    datetime64[ns]
LAST_RT                   float64
LAST_UPB                  float64
Delq.Status                object
MOD_FLAG                   object
Zero.Bal.Code             float64
ZB_DTE                     object
FCC_DTE                    object
dtype: object

In [16]:
# coding Delinquncy status into 0,1,2,3,4+ categories
df['Delq.Status_Coding'] = df['Delq.Status'].replace(['4','5','6','7','8','9','10','11','12','13',
                                                      '14','15','16','17','18','19','20','21','22'], '4+')
# Understand how many categories in Delinquency status
df['Delq.Status_Coding'].value_counts(dropna=False)

0      9187791
X        68114
1        52514
2         9927
4+        9424
3         4143
NaN        137
Name: Delq.Status_Coding, dtype: int64

In [15]:
df['Delq.Status'].value_counts(dropna=False)

0      9187791
X        68114
1        52514
2         9927
3         4143
4         2669
5         2016
6         1336
7          941
8          648
9          446
10         375
11         274
12         199
13         158
NaN        137
14         119
15          96
16          51
17          38
18          24
19          16
20          11
21           5
22           2
Name: Delq.Status, dtype: int64

In [17]:
df['Zero.Bal.Code'].value_counts(dropna=False)

NaN     9263800
1.0       67775
6.0         326
9.0          89
2.0          31
16.0         12
3.0          12
15.0          5
Name: Zero.Bal.Code, dtype: int64

In [18]:
#From the below result, we coukd know that only when delinquency status is unknown, Zero Balance code is denoted
df.groupby(['Delq.Status_Coding','Zero.Bal.Code']).size()

Delq.Status_Coding  Zero.Bal.Code
X                   1.0              67775
                    6.0                326
                    16.0                12
dtype: int64

In [19]:
# coding FCC as 'F'
df['FCC_DTE_Coding'] = df['FCC_DTE'].replace(['10/01/2019', '05/01/2019', '08/01/2019', '12/01/2018',
       '04/01/2019', '07/01/2019', '01/01/2019', '09/01/2019',
       '06/01/2019', '12/01/2019', '11/01/2018', '03/01/2019',
       '02/01/2019', '11/01/2019', '10/01/2018', '07/01/2018'],'F')

In [20]:
df['FCC_DTE_Coding'].value_counts(dropna=False)

NaN    9331924
F          126
Name: FCC_DTE_Coding, dtype: int64

In [21]:
# From below result, we know that Foreclosure only shows when loans denoted as 2.0, 3.0, 9.0. 

df.groupby(['FCC_DTE_Coding','Zero.Bal.Code']).size()

FCC_DTE_Coding  Zero.Bal.Code
F               2.0              31
                3.0               9
                9.0              86
dtype: int64

In [22]:
# In addition, when 'Delq.Status_Coding' shows, Foreclosure would not be come up. they totally exlusive. Combined with result above, 
# we could just denote 2.0, 3.0, 9.0. as 'F'
df.groupby(['FCC_DTE_Coding','Delq.Status_Coding']).size()

Series([], dtype: int64)

In [23]:
df['Zero.Bal.Code']=df['Zero.Bal.Code'].replace([2.0, 3.0, 9.0],'F')

In [24]:
df['Zero.Bal.Code'].value_counts(dropna=False)

NaN     9263800
1.0       67775
6.0         326
F           132
16.0         12
15.0          5
Name: Zero.Bal.Code, dtype: int64

In [25]:
df['Delq.Status_Coding'].value_counts()

0     9187791
X       68114
1       52514
2        9927
4+       9424
3        4143
Name: Delq.Status_Coding, dtype: int64

In [26]:
conditions = [
    df['Delq.Status_Coding']== '0' ,
    df['Delq.Status_Coding']== '1',
    df['Delq.Status_Coding']== '2',
    df['Delq.Status_Coding']== '3',
    df['Delq.Status_Coding']== '4+',
    ((pd.isna(df['Delq.Status_Coding']))& (df['Zero.Bal.Code']== 'F' )),
    ((df['Delq.Status_Coding']=='X' )& (df['Zero.Bal.Code']== 1.0 )),
    ((df['Delq.Status_Coding']=='X' )& (df['Zero.Bal.Code']== 6.0 )),
    ((df['Delq.Status_Coding']=='X' )& (df['Zero.Bal.Code']== 16.0 )),
    ((df['Delq.Status_Coding']=='X' )& (pd.isna(df['Zero.Bal.Code']))),
    ((pd.isna(df['Delq.Status_Coding']))& (df['Zero.Bal.Code']== 15.0))
]

In [27]:
choises = ['0','1','2','3','4+','F','P','Other','Other','Other','Other']

In [28]:
df['Loan_Status'] = np.select(condlist=conditions, choicelist = choises)

In [19]:
df['Loan_Status'].value_counts()

0        9187791
P          67775
1          52514
2           9927
4+          9424
3           4143
Other        344
F            132
Name: Loan_Status, dtype: int64

In [29]:
df = df.pivot('LOAN_ID','Monthly.Rpt.Prd','Loan_Status')

In [23]:
df.head

<bound method NDFrame.head of Monthly.Rpt.Prd 2018-01-01 2018-02-01 2018-03-01 2018-04-01 2018-05-01  \
LOAN_ID                                                                  
100001040173           NaN          0          0          0          0   
100002370993           NaN        NaN          0          0          0   
100005405807             0          0          0          0          0   
100008071646           NaN        NaN          0          0          0   
100010739040           NaN          0          0          0          0   
...                    ...        ...        ...        ...        ...   
999991303485             0          0          0          0          0   
999992661330           NaN          0          0          0          0   
999995056050           NaN          0          0          0          0   
999997685206           NaN        NaN          0          0          0   
999998783494           NaN          0          0          0          0   

Monthly

In [30]:
table_1 = pd.crosstab(df['2018-01-01'],df['2018-02-01'], margins=True)
table_1  

2018-02-01 00:00:00,0,1,2,Other,P,All
2018-01-01 00:00:00,,,,,,
0,153320,903,1,1,225,154450
1,84,7,9,0,0,100
All,293478,983,10,1,225,426206


In [59]:
table_1_P = pd.crosstab(df['2018-01-01'],df['2018-02-01'], normalize='index',margins=True)
table_1_P.style.background_gradient(cmap='Blues',axis=None) 


2018-02-01 00:00:00,0,1,2,Other,P
2018-01-01 00:00:00,,,,,
0,0.992684,0.005847,0.000006,0.000006,0.001457
1,0.840000,0.070000,0.090000,0.000000,0.000000
All,0.992585,0.005888,0.000065,0.000006,0.001456


In [38]:
table_2 = pd.crosstab(df['2018-02-01'],df['2018-03-01'], margins=True)
table_2  

2018-03-01 00:00:00,0,1,2,3,Other,P,All
2018-02-01 00:00:00,,,,,,,
0,291804,1081,0,1,1,590,293477
1,838,82,59,0,1,3,983
2,9,0,0,1,0,0,10
All,292651,1163,59,2,2,593,294470


In [60]:
table_2_P = pd.crosstab(df['2018-02-01'],df['2018-03-01'],normalize='index', margins=True)
table_2_P.style.background_gradient(cmap='Blues',axis=None)

2018-03-01 00:00:00,0,1,2,3,Other,P
2018-02-01 00:00:00,,,,,,
0,0.994299,0.003683,0.000000,0.000003,0.000003,0.002010
1,0.852492,0.083418,0.060020,0.000000,0.001017,0.003052
2,0.900000,0.000000,0.000000,0.100000,0.000000,0.000000
All,0.993823,0.003949,0.000200,0.000007,0.000007,0.002014


In [62]:
table_3 = pd.crosstab(df['2018-03-01'],df['2018-04-01'], margins=True)
table_3 

2018-04-01 00:00:00,0,1,2,3,4+,Other,P,All
2018-03-01 00:00:00,,,,,,,,
0,421931,1211,3,0,1,4,938,424088
1,950,163,117,0,0,0,5,1235
2,24,7,3,23,1,0,1,59
3,2,0,0,0,0,0,0,2
All,422907,1381,123,23,2,4,944,425384


In [64]:
table_3_P = pd.crosstab(df['2018-03-01'],df['2018-04-01'],normalize='index', margins=True)
table_3_P.style.background_gradient(cmap='Blues',axis=None)

2018-04-01 00:00:00,0,1,2,3,4+,Other,P
2018-03-01 00:00:00,,,,,,,
0,0.994914,0.002856,0.000007,0.000000,0.000002,0.000009,0.002212
1,0.769231,0.131984,0.094737,0.000000,0.000000,0.000000,0.004049
2,0.406780,0.118644,0.050847,0.389831,0.016949,0.000000,0.016949
3,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
All,0.994177,0.003246,0.000289,0.000054,0.000005,0.000009,0.002219


In [31]:
table_4 = pd.crosstab(df['2018-04-01'],df['2018-05-01'], margins=True)
table_4

2018-05-01 00:00:00,0,1,2,3,4+,Other,P,All
2018-04-01 00:00:00,,,,,,,,
0,420447,1080,5,0,0,5,1370,422907
1,1002,231,127,2,0,2,17,1381
2,36,7,14,66,0,0,0,123
3,11,0,1,1,10,0,0,23
4+,1,0,0,0,1,0,0,2
All,421497,1318,147,69,11,7,1387,424436


In [65]:
table_4_P = pd.crosstab(df['2018-04-01'],df['2018-05-01'],normalize='index', margins=True)
table_4_P.style.background_gradient(cmap='Blues',axis=None)

2018-05-01 00:00:00,0,1,2,3,4+,Other,P
2018-04-01 00:00:00,,,,,,,
0,0.994183,0.002554,0.000012,0.000000,0.000000,0.000012,0.003239
1,0.725561,0.167270,0.091962,0.001448,0.000000,0.001448,0.012310
2,0.292683,0.056911,0.113821,0.536585,0.000000,0.000000,0.000000
3,0.478261,0.000000,0.043478,0.043478,0.434783,0.000000,0.000000
4+,0.500000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000
All,0.993076,0.003105,0.000346,0.000163,0.000026,0.000016,0.003268


In [33]:
table_5 = pd.crosstab(df['2018-05-01'],df['2018-06-01'], margins=True)
table_5

2018-06-01 00:00:00,0,1,2,3,4+,Other,P,All
2018-05-01 00:00:00,,,,,,,,
0,418695,1186,6,2,1,9,1599,421498
1,878,304,119,1,0,0,16,1318
2,49,12,26,59,0,0,1,147
3,22,0,1,7,38,1,0,69
4+,2,0,1,0,8,0,0,11
All,419646,1502,153,69,47,10,1616,423043


In [67]:
table_5_P = pd.crosstab(df['2018-05-01'],df['2018-06-01'],normalize='index', margins=True)
table_5_P.style.background_gradient(cmap='Blues',axis=None)

2018-06-01 00:00:00,0,1,2,3,4+,Other,P
2018-05-01 00:00:00,,,,,,,
0,0.993350,0.002814,0.000014,0.000005,0.000002,0.000021,0.003794
1,0.666161,0.230653,0.090288,0.000759,0.000000,0.000000,0.012140
2,0.333333,0.081633,0.176871,0.401361,0.000000,0.000000,0.006803
3,0.318841,0.000000,0.014493,0.101449,0.550725,0.014493,0.000000
4+,0.181818,0.000000,0.090909,0.000000,0.727273,0.000000,0.000000
All,0.991970,0.003550,0.000362,0.000163,0.000111,0.000024,0.003820


In [73]:
table_6 = pd.crosstab(df['2018-06-01'],df['2018-07-01'], margins=True)
table_6

2018-07-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2018-06-01 00:00:00,,,,,,,,,
0,416578,1120,4,2,2,0,7,1933,419646
1,981,367,133,1,0,0,1,19,1502
2,36,22,32,60,2,0,0,1,153
3,13,0,5,5,41,1,1,3,69
4+,20,2,0,0,25,0,0,0,47
All,417628,1511,174,68,70,1,9,1956,421417


In [75]:
table_6_P = pd.crosstab(df['2018-06-01'],df['2018-07-01'],normalize='index',  margins=True)
table_6_P.style.background_gradient(cmap='Blues',axis=None)

2018-07-01 00:00:00,0,1,2,3,4+,F,Other,P
2018-06-01 00:00:00,,,,,,,,
0,0.992689,0.002669,0.000010,0.000005,0.000005,0.000000,0.000017,0.004606
1,0.653129,0.244341,0.088549,0.000666,0.000000,0.000000,0.000666,0.012650
2,0.235294,0.143791,0.209150,0.392157,0.013072,0.000000,0.000000,0.006536
3,0.188406,0.000000,0.072464,0.072464,0.594203,0.014493,0.014493,0.043478
4+,0.425532,0.042553,0.000000,0.000000,0.531915,0.000000,0.000000,0.000000
All,0.991009,0.003586,0.000413,0.000161,0.000166,0.000002,0.000021,0.004641


In [76]:
table_7 = pd.crosstab(df['2018-07-01'],df['2018-08-01'], margins=True)
table_7

2018-08-01 00:00:00,0,1,2,3,4+,Other,P,All
2018-07-01 00:00:00,,,,,,,,
0,414416,973,5,2,1,20,2211,417628
1,931,408,142,1,0,0,29,1511
2,35,24,47,63,0,0,5,174
3,12,5,6,5,40,0,0,68
4+,9,1,2,1,54,1,2,70
All,415403,1411,202,72,95,21,2247,419451


In [77]:
table_7_P = pd.crosstab(df['2018-07-01'],df['2018-08-01'],normalize='index', margins=True)
table_7_P.style.background_gradient(cmap='Blues', axis=None)

2018-08-01 00:00:00,0,1,2,3,4+,Other,P
2018-07-01 00:00:00,,,,,,,
0,0.992309,0.002330,0.000012,0.000005,0.000002,0.000048,0.005294
1,0.616148,0.270020,0.093977,0.000662,0.000000,0.000000,0.019193
2,0.201149,0.137931,0.270115,0.362069,0.000000,0.000000,0.028736
3,0.176471,0.073529,0.088235,0.073529,0.588235,0.000000,0.000000
4+,0.128571,0.014286,0.028571,0.014286,0.771429,0.014286,0.028571
All,0.990349,0.003364,0.000482,0.000172,0.000226,0.000050,0.005357


In [80]:
table_8 = pd.crosstab(df['2018-08-01'],df['2018-09-01'], margins=True)
table_8

2018-09-01 00:00:00,0,1,2,3,4+,Other,P,All
2018-08-01 00:00:00,,,,,,,,
0,411776,1684,10,3,1,23,1906,415403
1,674,506,213,3,0,0,15,1411
2,26,24,58,87,1,0,6,202
3,7,3,11,12,38,0,1,72
4+,5,2,2,0,84,0,2,95
All,412488,2219,294,105,124,23,1930,417183


In [81]:
table_8_P = pd.crosstab(df['2018-08-01'],df['2018-09-01'],normalize='index', margins=True)
table_8_P.style.background_gradient(cmap='Blues', axis=None)

2018-09-01 00:00:00,0,1,2,3,4+,Other,P
2018-08-01 00:00:00,,,,,,,
0,0.991269,0.004054,0.000024,0.000007,0.000002,0.000055,0.004588
1,0.477675,0.358611,0.150957,0.002126,0.000000,0.000000,0.010631
2,0.128713,0.118812,0.287129,0.430693,0.004950,0.000000,0.029703
3,0.097222,0.041667,0.152778,0.166667,0.527778,0.000000,0.013889
4+,0.052632,0.021053,0.021053,0.000000,0.884211,0.000000,0.021053
All,0.988746,0.005319,0.000705,0.000252,0.000297,0.000055,0.004626


In [37]:
table_9 = pd.crosstab(df['2018-09-01'],df['2018-10-01'], margins=True)
table_9

2018-10-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2018-09-01 00:00:00,,,,,,,,,
0,409021,1268,5,1,1,0,42,2150,412488
1,1279,674,229,2,0,0,2,33,2219
2,59,36,98,94,3,0,0,4,294
3,8,6,9,21,59,0,0,2,105
4+,5,0,4,2,110,2,0,1,124
All,410372,1984,345,120,173,2,44,2190,415230


In [83]:
table_9_P = pd.crosstab(df['2018-09-01'],df['2018-10-01'], normalize = 'index', margins=True)
table_9_P.style.background_gradient(cmap='Blues', axis=None)

2018-10-01 00:00:00,0,1,2,3,4+,F,Other,P
2018-09-01 00:00:00,,,,,,,,
0,0.991595,0.003074,0.000012,0.000002,0.000002,0.000000,0.000102,0.005212
1,0.576386,0.303740,0.103200,0.000901,0.000000,0.000000,0.000901,0.014872
2,0.200680,0.122449,0.333333,0.319728,0.010204,0.000000,0.000000,0.013605
3,0.076190,0.057143,0.085714,0.200000,0.561905,0.000000,0.000000,0.019048
4+,0.040323,0.000000,0.032258,0.016129,0.887097,0.016129,0.000000,0.008065
All,0.988300,0.004778,0.000831,0.000289,0.000417,0.000005,0.000106,0.005274


In [38]:
table_10 = pd.crosstab(df['2018-10-01'],df['2018-11-01'], margins=True)
table_10

2018-11-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2018-10-01 00:00:00,,,,,,,,,
0,407086,1396,11,1,0,0,39,1839,410372
1,992,695,273,2,0,0,0,22,1984
2,54,57,110,119,1,0,0,4,345
3,10,7,7,26,69,0,0,1,120
4+,14,3,1,3,146,3,2,1,173
All,408156,2158,402,151,216,3,41,1867,412994


In [84]:
table_10_P = pd.crosstab(df['2018-10-01'],df['2018-11-01'], normalize = 'index', margins=True)
table_10_P.style.background_gradient(cmap='Blues', axis=None)

2018-11-01 00:00:00,0,1,2,3,4+,F,Other,P
2018-10-01 00:00:00,,,,,,,,
0,0.991993,0.003402,0.000027,0.000002,0.000000,0.000000,0.000095,0.004481
1,0.500000,0.350302,0.137601,0.001008,0.000000,0.000000,0.000000,0.011089
2,0.156522,0.165217,0.318841,0.344928,0.002899,0.000000,0.000000,0.011594
3,0.083333,0.058333,0.058333,0.216667,0.575000,0.000000,0.000000,0.008333
4+,0.080925,0.017341,0.005780,0.017341,0.843931,0.017341,0.011561,0.005780
All,0.988286,0.005225,0.000973,0.000366,0.000523,0.000007,0.000099,0.004521


In [39]:
table_11 = pd.crosstab(df['2018-11-01'],df['2018-12-01'], margins=True)
table_11

2018-12-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2018-11-01 00:00:00,,,,,,,,,
0,404447,1867,6,0,2,0,46,1788,408156
1,1071,726,328,8,0,1,2,22,2158
2,55,42,114,179,2,0,0,10,402
3,10,4,11,24,101,1,0,0,151
4+,11,2,0,1,198,1,0,3,216
All,405594,2641,459,212,303,3,48,1823,411083


In [85]:
table_11_P = pd.crosstab(df['2018-11-01'],df['2018-12-01'], normalize = 'index', margins=True)
table_11_P.style.background_gradient(cmap='Blues', axis=None)

2018-12-01 00:00:00,0,1,2,3,4+,F,Other,P
2018-11-01 00:00:00,,,,,,,,
0,0.990913,0.004574,0.000015,0.000000,0.000005,0.000000,0.000113,0.004381
1,0.496293,0.336423,0.151993,0.003707,0.000000,0.000463,0.000927,0.010195
2,0.136816,0.104478,0.283582,0.445274,0.004975,0.000000,0.000000,0.024876
3,0.066225,0.026490,0.072848,0.158940,0.668874,0.006623,0.000000,0.000000
4+,0.050926,0.009259,0.000000,0.004630,0.916667,0.004630,0.000000,0.013889
All,0.986647,0.006424,0.001117,0.000516,0.000737,0.000007,0.000117,0.004435


In [40]:
table_12 = pd.crosstab(df['2018-12-01'],df['2019-01-01'], margins=True)
table_12

2019-01-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2018-12-01 00:00:00,,,,,,,,,
0,402318,1450,6,3,1,0,29,1787,405594
1,1431,826,347,3,1,0,0,33,2641
2,68,56,178,150,0,0,1,6,459
3,26,11,20,36,114,0,1,4,212
4+,12,0,3,7,271,7,3,0,303
All,403855,2343,554,199,387,7,34,1830,409209


In [86]:
table_12_P = pd.crosstab(df['2018-12-01'],df['2019-01-01'], normalize = 'index', margins=True)
table_12_P.style.background_gradient(cmap='Blues', axis=None)

2019-01-01 00:00:00,0,1,2,3,4+,F,Other,P
2018-12-01 00:00:00,,,,,,,,
0,0.991923,0.003575,0.000015,0.000007,0.000002,0.000000,0.000072,0.004406
1,0.541840,0.312760,0.131390,0.001136,0.000379,0.000000,0.000000,0.012495
2,0.148148,0.122004,0.387800,0.326797,0.000000,0.000000,0.002179,0.013072
3,0.122642,0.051887,0.094340,0.169811,0.537736,0.000000,0.004717,0.018868
4+,0.039604,0.000000,0.009901,0.023102,0.894389,0.023102,0.009901,0.000000
All,0.986916,0.005726,0.001354,0.000486,0.000946,0.000017,0.000083,0.004472


In [41]:
table_13 = pd.crosstab(df['2019-01-01'],df['2019-02-01'], margins=True)
table_13

2019-02-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-01-01 00:00:00,,,,,,,,,
0,399529,2233,5,0,1,0,19,2068,403855
1,1122,846,343,3,3,0,0,26,2343
2,106,86,163,190,2,0,0,7,554
3,25,5,16,33,115,0,0,5,199
4+,18,3,1,2,356,5,0,2,387
All,400800,3173,528,228,477,5,19,2108,407338


In [88]:
table_13_P = pd.crosstab(df['2019-01-01'],df['2019-02-01'], normalize = 'index', margins=True)
table_13_P.style.background_gradient(cmap='Blues', axis=None)

2019-02-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-01-01 00:00:00,,,,,,,,
0,0.989288,0.005529,0.000012,0.000000,0.000002,0.000000,0.000047,0.005121
1,0.478873,0.361076,0.146394,0.001280,0.001280,0.000000,0.000000,0.011097
2,0.191336,0.155235,0.294224,0.342960,0.003610,0.000000,0.000000,0.012635
3,0.125628,0.025126,0.080402,0.165829,0.577889,0.000000,0.000000,0.025126
4+,0.046512,0.007752,0.002584,0.005168,0.919897,0.012920,0.000000,0.005168
All,0.983949,0.007790,0.001296,0.000560,0.001171,0.000012,0.000047,0.005175


In [42]:
table_14 = pd.crosstab(df['2019-02-01'],df['2019-03-01'], margins=True)
table_14

2019-03-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-02-01 00:00:00,,,,,,,,,
0,396458,1645,4,1,0,0,18,2674,400800
1,1832,957,319,6,1,0,1,57,3173
2,115,97,163,139,3,0,0,11,528
3,20,5,23,44,133,0,0,3,228
4+,32,4,2,9,416,3,0,11,477
All,398457,2708,511,199,553,3,19,2756,405206


In [89]:
table_14_P = pd.crosstab(df['2019-02-01'],df['2019-03-01'], normalize = 'index', margins=True)
table_14_P.style.background_gradient(cmap='Blues', axis=None)

2019-03-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-02-01 00:00:00,,,,,,,,
0,0.989167,0.004104,0.000010,0.000002,0.000000,0.000000,0.000045,0.006672
1,0.577372,0.301607,0.100536,0.001891,0.000315,0.000000,0.000315,0.017964
2,0.217803,0.183712,0.308712,0.263258,0.005682,0.000000,0.000000,0.020833
3,0.087719,0.021930,0.100877,0.192982,0.583333,0.000000,0.000000,0.013158
4+,0.067086,0.008386,0.004193,0.018868,0.872117,0.006289,0.000000,0.023061
All,0.983344,0.006683,0.001261,0.000491,0.001365,0.000007,0.000047,0.006801


In [90]:
table_15 = pd.crosstab(df['2019-03-01'],df['2019-04-01'], margins=True)
table_15

2019-04-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-03-01 00:00:00,,,,,,,,,
0,393758,1517,5,3,2,0,5,3167,398457
1,1464,881,323,2,0,0,0,38,2708
2,85,88,176,153,2,0,1,6,511
3,20,7,15,39,113,0,0,5,199
4+,32,4,4,11,477,11,1,13,553
All,395359,2497,523,208,594,11,7,3229,402428


In [91]:
table_15_P = pd.crosstab(df['2019-03-01'],df['2019-04-01'], normalize = 'index', margins=True)
table_15_P.style.background_gradient(cmap='Blues', axis=None)

2019-04-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-03-01 00:00:00,,,,,,,,
0,0.988207,0.003807,0.000013,0.000008,0.000005,0.000000,0.000013,0.007948
1,0.540620,0.325332,0.119276,0.000739,0.000000,0.000000,0.000000,0.014032
2,0.166341,0.172211,0.344423,0.299413,0.003914,0.000000,0.001957,0.011742
3,0.100503,0.035176,0.075377,0.195980,0.567839,0.000000,0.000000,0.025126
4+,0.057866,0.007233,0.007233,0.019892,0.862568,0.019892,0.001808,0.023508
All,0.982434,0.006205,0.001300,0.000517,0.001476,0.000027,0.000017,0.008024


In [92]:
table_16 = pd.crosstab(df['2019-04-01'],df['2019-05-01'], margins=True)
table_16

2019-05-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-04-01 00:00:00,,,,,,,,,
0,390140,1383,6,0,0,0,4,3826,395359
1,1218,911,324,2,0,1,0,41,2497
2,77,89,179,170,2,0,0,6,523
3,34,5,16,43,104,0,0,6,208
4+,43,8,5,5,514,9,1,9,594
All,391512,2396,530,220,620,10,5,3888,399181


In [45]:
table_17 = pd.crosstab(df['2019-05-01'],df['2019-06-01'], margins=True)
table_17

2019-06-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-05-01 00:00:00,,,,,,,,,
0,385732,2186,9,1,1,0,2,3581,391512
1,942,976,429,5,3,0,0,41,2396
2,71,72,181,190,5,0,1,10,530
3,17,4,20,48,127,0,0,4,220
4+,47,8,3,8,542,6,1,5,620
All,386809,3246,642,252,678,6,4,3641,395278


In [93]:
table_17_P = pd.crosstab(df['2019-05-01'],df['2019-06-01'], normalize = 'index', margins=True)
table_17_P.style.background_gradient(cmap='Blues', axis=None)

2019-06-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-05-01 00:00:00,,,,,,,,
0,0.985237,0.005583,0.000023,0.000003,0.000003,0.000000,0.000005,0.009147
1,0.393155,0.407346,0.179048,0.002087,0.001252,0.000000,0.000000,0.017112
2,0.133962,0.135849,0.341509,0.358491,0.009434,0.000000,0.001887,0.018868
3,0.077273,0.018182,0.090909,0.218182,0.577273,0.000000,0.000000,0.018182
4+,0.075806,0.012903,0.004839,0.012903,0.874194,0.009677,0.001613,0.008065
All,0.978575,0.008212,0.001624,0.000638,0.001715,0.000015,0.000010,0.009211


In [94]:
table_18 = pd.crosstab(df['2019-06-01'],df['2019-07-01'], margins=True)
table_18

2019-07-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-06-01 00:00:00,,,,,,,,,
0,380652,1554,5,1,1,0,7,4589,386809
1,1613,1131,431,1,0,0,0,70,3246
2,94,116,220,199,1,0,0,12,642
3,18,13,17,68,130,0,0,6,252
4+,65,8,1,10,568,9,4,13,678
All,382442,2822,674,279,700,9,11,4690,391627


In [95]:
table_18_P = pd.crosstab(df['2019-06-01'],df['2019-07-01'], normalize = 'index', margins=True)
table_18_P.style.background_gradient(cmap='Blues', axis=None)

2019-07-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-06-01 00:00:00,,,,,,,,
0,0.984083,0.004017,0.000013,0.000003,0.000003,0.000000,0.000018,0.011864
1,0.496919,0.348429,0.132779,0.000308,0.000000,0.000000,0.000000,0.021565
2,0.146417,0.180685,0.342679,0.309969,0.001558,0.000000,0.000000,0.018692
3,0.071429,0.051587,0.067460,0.269841,0.515873,0.000000,0.000000,0.023810
4+,0.095870,0.011799,0.001475,0.014749,0.837758,0.013274,0.005900,0.019174
All,0.976547,0.007206,0.001721,0.000712,0.001787,0.000023,0.000028,0.011976


In [47]:
table_19= pd.crosstab(df['2019-07-01'],df['2019-08-01'], margins=True)
table_19

2019-08-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-07-01 00:00:00,,,,,,,,,
0,375734,1508,8,4,1,0,5,5182,382442
1,1298,1050,428,3,1,0,0,42,2822
2,104,123,205,229,1,0,0,12,674
3,17,11,27,56,159,0,0,9,279
4+,61,8,3,9,589,14,2,14,700
All,377214,2700,671,301,751,14,7,5259,386917


In [96]:
table_19_P = pd.crosstab(df['2019-07-01'],df['2019-08-01'], normalize = 'index', margins=True)
table_19_P.style.background_gradient(cmap='Blues', axis=None)

2019-08-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-07-01 00:00:00,,,,,,,,
0,0.982460,0.003943,0.000021,0.000010,0.000003,0.000000,0.000013,0.013550
1,0.459957,0.372077,0.151665,0.001063,0.000354,0.000000,0.000000,0.014883
2,0.154303,0.182493,0.304154,0.339763,0.001484,0.000000,0.000000,0.017804
3,0.060932,0.039427,0.096774,0.200717,0.569892,0.000000,0.000000,0.032258
4+,0.087143,0.011429,0.004286,0.012857,0.841429,0.020000,0.002857,0.020000
All,0.974922,0.006978,0.001734,0.000778,0.001941,0.000036,0.000018,0.013592


In [97]:
table_20= pd.crosstab(df['2019-08-01'],df['2019-09-01'], margins=True)
table_20

2019-09-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-08-01 00:00:00,,,,,,,,,
0,369458,1814,8,1,2,0,1,5930,377214
1,1077,1117,458,4,0,0,4,40,2700
2,86,103,224,238,3,0,0,17,671
3,33,11,19,73,162,0,0,3,301
4+,57,6,4,10,655,8,1,10,751
All,370711,3051,713,326,822,8,6,6000,381637


In [98]:
table_20_P = pd.crosstab(df['2019-08-01'],df['2019-09-01'], normalize = 'index', margins=True)
table_20_P.style.background_gradient(cmap='Blues', axis=None)

2019-09-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-08-01 00:00:00,,,,,,,,
0,0.979439,0.004809,0.000021,0.000003,0.000005,0.000000,0.000003,0.015721
1,0.398889,0.413704,0.169630,0.001481,0.000000,0.000000,0.001481,0.014815
2,0.128167,0.153502,0.333830,0.354694,0.004471,0.000000,0.000000,0.025335
3,0.109635,0.036545,0.063123,0.242525,0.538206,0.000000,0.000000,0.009967
4+,0.075899,0.007989,0.005326,0.013316,0.872170,0.010652,0.001332,0.013316
All,0.971371,0.007995,0.001868,0.000854,0.002154,0.000021,0.000016,0.015722


In [99]:
table_21= pd.crosstab(df['2019-09-01'],df['2019-10-01'], margins=True)
table_21

2019-10-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-09-01 00:00:00,,,,,,,,,
0,362518,1526,11,2,0,0,2,6652,370711
1,1366,1149,435,1,0,0,1,99,3051
2,103,114,238,239,1,0,1,17,713
3,30,14,28,81,170,1,0,2,326
4+,66,5,7,7,698,19,1,18,821
All,364083,2808,719,330,869,20,5,6788,375622


In [100]:
table_21_P = pd.crosstab(df['2019-09-01'],df['2019-10-01'], normalize = 'index', margins=True)
table_21_P.style.background_gradient(cmap='Blues', axis=None)

2019-10-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-09-01 00:00:00,,,,,,,,
0,0.977899,0.004116,0.000030,0.000005,0.000000,0.000000,0.000005,0.017944
1,0.447722,0.376598,0.142576,0.000328,0.000000,0.000000,0.000328,0.032448
2,0.144460,0.159888,0.333801,0.335203,0.001403,0.000000,0.001403,0.023843
3,0.092025,0.042945,0.085890,0.248466,0.521472,0.003067,0.000000,0.006135
4+,0.080390,0.006090,0.008526,0.008526,0.850183,0.023143,0.001218,0.021924
All,0.969280,0.007476,0.001914,0.000879,0.002313,0.000053,0.000013,0.018071


In [50]:
table_22= pd.crosstab(df['2019-10-01'],df['2019-11-01'], margins=True)
table_22

2019-08-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-07-01 00:00:00,,,,,,,,,
0,375734,1508,8,4,1,0,5,5182,382442
1,1298,1050,428,3,1,0,0,42,2822
2,104,123,205,229,1,0,0,12,674
3,17,11,27,56,159,0,0,9,279
4+,61,8,3,9,589,14,2,14,700
All,377214,2700,671,301,751,14,7,5259,386917


In [101]:
table_22_P = pd.crosstab(df['2019-10-01'],df['2019-11-01'], normalize = 'index', margins=True)
table_22_P.style.background_gradient(cmap='Blues', axis=None)

2019-11-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-10-01 00:00:00,,,,,,,,
0,0.980032,0.005202,0.000027,0.000005,0.000000,0.000000,0.000005,0.014727
1,0.398148,0.422009,0.161681,0.001425,0.000000,0.000000,0.000000,0.016738
2,0.119611,0.158554,0.328234,0.378303,0.002782,0.000000,0.000000,0.012517
3,0.069697,0.048485,0.078788,0.227273,0.560606,0.000000,0.000000,0.015152
4+,0.067894,0.012658,0.003452,0.010357,0.864212,0.019563,0.001151,0.020713
All,0.970961,0.008731,0.001977,0.000982,0.002543,0.000046,0.000008,0.014753


In [102]:
table_23= pd.crosstab(df['2019-11-01'],df['2019-12-01'], margins=True)
table_23

2019-12-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-11-01 00:00:00,,,,,,,,,
0,351126,1700,5,4,0,0,8,5256,358099
1,1403,1267,465,3,1,0,1,80,3220
2,101,108,263,239,0,0,4,14,729
3,33,16,28,90,186,0,1,8,362
4+,79,16,4,12,805,13,0,9,938
All,352742,3107,765,348,992,13,14,5367,363348


In [103]:
table_23_P = pd.crosstab(df['2019-11-01'],df['2019-12-01'], normalize = 'index', margins=True)
table_23_P.style.background_gradient(cmap='Blues', axis=None)

2019-12-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-11-01 00:00:00,,,,,,,,
0,0.980528,0.004747,0.000014,0.000011,0.000000,0.000000,0.000022,0.014678
1,0.435714,0.393478,0.144410,0.000932,0.000311,0.000000,0.000311,0.024845
2,0.138546,0.148148,0.360768,0.327846,0.000000,0.000000,0.005487,0.019204
3,0.091160,0.044199,0.077348,0.248619,0.513812,0.000000,0.002762,0.022099
4+,0.084222,0.017058,0.004264,0.012793,0.858209,0.013859,0.000000,0.009595
All,0.970810,0.008551,0.002105,0.000958,0.002730,0.000036,0.000039,0.014771


In [104]:
table_HY1= pd.crosstab(df['2018-01-01'],df['2018-06-01'], margins=True)
table_HY1

2018-06-01 00:00:00,0,1,2,3,4+,Other,P,All
2018-01-01 00:00:00,,,,,,,,
0,151481,596,73,27,18,5,699,152899
1,95,2,0,1,0,0,1,99
All,151576,598,73,28,18,5,700,152998


In [106]:
table_HY1_P = pd.crosstab(df['2018-01-01'],df['2018-06-01'], normalize = 'index', margins=True)
table_HY1_P.style.background_gradient(cmap='Blues', axis=None)

2018-06-01 00:00:00,0,1,2,3,4+,Other,P
2018-01-01 00:00:00,,,,,,,
0,0.990726,0.003898,0.000477,0.000177,0.000118,0.000033,0.004572
1,0.959596,0.020202,0.000000,0.010101,0.000000,0.000000,0.010101
All,0.990706,0.003909,0.000477,0.000183,0.000118,0.000033,0.004575


In [107]:
table_HY2= pd.crosstab(df['2018-06-01'],df['2018-12-01'], margins=True)
table_HY2

2018-12-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2018-06-01 00:00:00,,,,,,,,,
0,404494,2387,366,158,140,2,47,1807,409401
1,995,229,78,41,76,0,0,14,1433
2,53,18,12,10,47,1,1,2,144
3,24,7,1,3,28,0,0,0,63
4+,28,0,2,0,12,0,0,0,42
All,405594,2641,459,212,303,3,48,1823,411083


In [108]:
table_HY2_P = pd.crosstab(df['2018-06-01'],df['2018-12-01'], normalize = 'index', margins=True)
table_HY2_P.style.background_gradient(cmap='Blues', axis=None)

2018-12-01 00:00:00,0,1,2,3,4+,F,Other,P
2018-06-01 00:00:00,,,,,,,,
0,0.988014,0.005830,0.000894,0.000386,0.000342,0.000005,0.000115,0.004414
1,0.694348,0.159805,0.054431,0.028611,0.053036,0.000000,0.000000,0.009770
2,0.368056,0.125000,0.083333,0.069444,0.326389,0.006944,0.006944,0.013889
3,0.380952,0.111111,0.015873,0.047619,0.444444,0.000000,0.000000,0.000000
4+,0.666667,0.000000,0.047619,0.000000,0.285714,0.000000,0.000000,0.000000
All,0.986647,0.006424,0.001117,0.000516,0.000737,0.000007,0.000117,0.004435


In [109]:
table_HY3= pd.crosstab(df['2019-01-01'],df['2019-06-01'], margins=True)
table_HY3

2019-06-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-01-01 00:00:00,,,,,,,,,
0,385203,2573,348,124,100,0,2,3625,391975
1,1319,521,214,75,117,0,0,11,2257
2,146,102,60,36,173,2,1,3,523
3,56,17,13,11,85,0,0,0,182
4+,85,33,7,6,203,4,1,2,341
All,386809,3246,642,252,678,6,4,3641,395278


In [110]:
table_HY3_P= pd.crosstab(df['2019-01-01'],df['2019-06-01'], normalize = 'index',margins=True)
table_HY3_P.style.background_gradient(cmap='Blues', axis=None)

2019-06-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-01-01 00:00:00,,,,,,,,
0,0.982723,0.006564,0.000888,0.000316,0.000255,0.000000,0.000005,0.009248
1,0.584404,0.230837,0.094816,0.033230,0.051839,0.000000,0.000000,0.004874
2,0.279159,0.195029,0.114723,0.068834,0.330784,0.003824,0.001912,0.005736
3,0.307692,0.093407,0.071429,0.060440,0.467033,0.000000,0.000000,0.000000
4+,0.249267,0.096774,0.020528,0.017595,0.595308,0.011730,0.002933,0.005865
All,0.978575,0.008212,0.001624,0.000638,0.001715,0.000015,0.000010,0.009211


In [111]:
table_HY4= pd.crosstab(df['2019-06-01'],df['2019-12-01'], margins=True)
table_HY4

2019-12-01 00:00:00,0,1,2,3,4+,F,Other,P,All
2019-06-01 00:00:00,,,,,,,,,
0,350534,2266,396,170,222,0,8,5332,358928
1,1758,674,252,102,222,3,3,25,3039
2,197,98,76,43,176,1,3,4,598
3,79,22,18,12,99,0,0,0,230
4+,174,47,23,21,274,9,0,6,554
All,352742,3107,765,348,993,13,14,5367,363349


In [112]:
table_HY4_P= pd.crosstab(df['2019-06-01'],df['2019-12-01'], normalize = 'index',margins=True)
table_HY4_P.style.background_gradient(cmap='Blues', axis=None)

2019-12-01 00:00:00,0,1,2,3,4+,F,Other,P
2019-06-01 00:00:00,,,,,,,,
0,0.976614,0.006313,0.001103,0.000474,0.000619,0.000000,0.000022,0.014855
1,0.578480,0.221783,0.082922,0.033564,0.073050,0.000987,0.000987,0.008226
2,0.329431,0.163880,0.127090,0.071906,0.294314,0.001672,0.005017,0.006689
3,0.343478,0.095652,0.078261,0.052174,0.430435,0.000000,0.000000,0.000000
4+,0.314079,0.084838,0.041516,0.037906,0.494585,0.016245,0.000000,0.010830
All,0.970808,0.008551,0.002105,0.000958,0.002733,0.000036,0.000039,0.014771
